In [1]:
%matplotlib inline

import itertools
import spacy
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import keras.backend as K
import tensorflow as tf

from collections import defaultdict, Counter

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Embedding, Dropout, Reshape, TimeDistributed
from keras.utils import to_categorical, plot_model

from keras_tqdm import TQDMNotebookCallback, TQDMCallback

from pprint import pprint

from scipy import stats
from sklearn.metrics import cohen_kappa_score, accuracy_score, classification_report
from sklearn.metrics import precision_recall_fscore_support 
from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit

from tqdm import tqdm_notebook as tqdm

from IPython.core.debugger import set_trace

Using TensorFlow backend.


In [2]:
# Check if GPU enabled

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/cpu:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 3553982065031890252, name: "/gpu:0"
 device_type: "GPU"
 memory_limit: 10968950375
 locality {
   bus_id: 1
 }
 incarnation: 3989715045338005209
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:04:00.0", name: "/gpu:1"
 device_type: "GPU"
 memory_limit: 10968950375
 locality {
   bus_id: 1
 }
 incarnation: 13143969203012567611
 physical_device_desc: "device: 1, name: GeForce GTX 1080 Ti, pci bus id: 0000:05:00.0", name: "/gpu:2"
 device_type: "GPU"
 memory_limit: 485883904
 locality {
   bus_id: 1
 }
 incarnation: 13343466826744085488
 physical_device_desc: "device: 2, name: GeForce GTX 1080 Ti, pci bus id: 0000:08:00.0", name: "/gpu:3"
 device_type: "GPU"
 memory_limit: 6748274688
 locality {
   bus_id: 1
 }
 incarnation: 12595380018081630311
 physical_device_desc: "device: 3, name: GeForce GTX 1080 Ti, pci bus id: 0000:09:00.0", name: "/gpu:4"
 device_type: "GPU"
 m

# Data Processing

## Twitter

In [2]:
with open('/usr0/home/mamille2/twitter/data/huang2016/huang2016_train.aligned.pkl', 'rb') as f:
#     twitter_texts, twitter_tags, twitter_histories = pickle.load(f)
    twitter_texts, twitter_tags, _ = pickle.load(f)
#     , twitter_tags, _ = pickle.load(f)
    
with open('/usr0/home/mamille2/twitter/data/huang2016/huang2016_valid.aligned.pkl', 'rb') as f:
#     dev_texts, dev_tags, dev_histories = pickle.load(f)
    dev_texts, dev_tags, _ = pickle.load(f)
#     _, dev_tags, _ = pickle.load(f)
    
with open('/usr0/home/mamille2/twitter/data/huang2016/huang2016_test.aligned.pkl', 'rb') as f:
#     _, test_tags, _ = pickle.load(f)
    test_texts, test_tags, _ = pickle.load(f)

In [3]:
# Extract character set

chars = set()
for text in twitter_texts:
    chars.update(set(text))
    
chars = sorted(chars)
chars = ['<PAD>', '<UNK>'] + chars
print ('{} unique characters.'.format(len(chars)))

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

1802 unique characters.


In [4]:
MAX_LENGTH = 140

In [5]:
# Vectorize tweets

# train
train_X = np.zeros((len(twitter_texts), MAX_LENGTH), dtype=np.bool)
for i, text in tqdm(enumerate(twitter_texts), total=len(twitter_texts)):
    for t, char in enumerate(text):
        if t >= MAX_LENGTH:
            break
        train_X[i, t] = char_indices[char]
        
# dev
dev_X = np.zeros((len(dev_texts), MAX_LENGTH), dtype=np.bool)
for i, text in tqdm(enumerate(dev_texts), total=len(dev_texts)):
    for t, char in enumerate(text):
        if t >= MAX_LENGTH:
            break
        dev_X[i, t] = char_indices.get(char, '<UNK>')
        
# test
test_X = np.zeros((len(test_texts), MAX_LENGTH), dtype=np.bool)
for i, text in tqdm(enumerate(test_texts), total=len(test_texts)):
    for t, char in enumerate(text):
        if t >= MAX_LENGTH:
            break
        test_X[i, t] = char_indices.get(char, '<UNK>')

HBox(children=(IntProgress(value=0, max=212855), HTML(value='')))

HBox(children=(IntProgress(value=0, max=25817), HTML(value='')))

HBox(children=(IntProgress(value=0, max=19614), HTML(value='')))

In [6]:
# Extract tag set

tag_set = set()
for t in twitter_tags:
    tag_set.update(set(t))
    
tag_set = sorted(tag_set)
print ('{} unique tags.'.format(len(tag_set)))

# Select top 3883 tags like Huang+ 2016
twitter_tags_c = Counter([t for ts in twitter_tags for t in ts]).most_common(3883)
top_tags = set([t for t,_ in twitter_tags_c])

tag_indices = dict((t, i) for i, t in enumerate(top_tags))
indices_tag = dict((i, t) for i, t in enumerate(top_tags))
print ('Selected {} tags.'.format(len(top_tags)))

37399 unique tags.
Selected 3883 tags.


In [7]:
# Vectorize tags

# train
train_y = np.zeros((len(twitter_tags), len(top_tags)), dtype=np.bool)
for i, tags in tqdm(enumerate(twitter_tags), total=len(twitter_tags)):
    for tag in tags:
        if tag in top_tags:
            train_y[i, tag_indices[tag]] = 1
        
# dev
dev_y = np.zeros((len(dev_tags), len(top_tags)), dtype=np.bool)
for i, tags in tqdm(enumerate(dev_tags), total=len(dev_tags)):
    for tag in tags:
        if tag in top_tags:
            dev_y[i, tag_indices[tag]] = 1
            
# test
test_y = np.zeros((len(test_tags), len(top_tags)), dtype=np.bool)
for i, tags in tqdm(enumerate(test_tags), total=len(test_tags)):
    for tag in tags:
        if tag in top_tags:
            test_y[i, tag_indices[tag]] = 1

HBox(children=(IntProgress(value=0, max=212855), HTML(value='')))

HBox(children=(IntProgress(value=0, max=25817), HTML(value='')))

HBox(children=(IntProgress(value=0, max=19614), HTML(value='')))

In [9]:
train_X.shape, train_y.shape, dev_X.shape, dev_y.shape, test_X.shape, test_y.shape

((212855, 140),
 (212855, 3883),
 (25817, 140),
 (25817, 3883),
 (19614, 140),
 (19614, 3883))

# Build Model

## BiLSTM Baseline

In [8]:
model = Sequential()
model.add(Embedding(len(chars) + 1,
                    64,
                    input_length=MAX_LENGTH,
                    trainable=True,
                    mask_zero=True))
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(.5))
model.add(Dense(len(top_tags), activation='softmax'))
print (model.output_shape)

(None, 3883)


In [9]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy'])

In [42]:
filepath="./weights.twitter.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_categorical_accuracy', verbose=1, save_best_only=True, mode='max')
early_stop = EarlyStopping(monitor='val_categorical_accuracy', patience=3)
callbacks_list = [checkpoint, early_stop]

In [46]:
model.fit(train_X,
              train_y,
              validation_data=(dev_X, dev_y),
              shuffle=True,
              batch_size=32,
              epochs=10,
              verbose=True,
              callbacks=callbacks_list)

print('BiLSTM Baseline Training complete!')

Train on 212855 samples, validate on 25817 samples
Epoch 1/10
 24608/212855 [==>...........................] - ETA: 1804s - loss: 8.9141 - categorical_accuracy: 0.0040

KeyboardInterrupt: 

# Evaluate

In [10]:
model.load_weights('/usr0/home/mamille2/twitter/models/bilstm_weights.hdf5')

In [11]:
pred = model.predict(test_X)

In [12]:
p = np.argmax(pred, axis=1)
y = np.argmax(test_y, axis=1)
p.shape, y.shape

((19614,), (19614,))

In [13]:
accuracy_score(p, y) # 0.1827 for trec2011 128-hidden unit baseline, 0.000153 for 32-hidden unit on huang2016 data

5.0983991026817579e-05

In [17]:
test_X.shape, test_y.shape

((19614, 140), (19614, 3883))

In [24]:
pred[0]

array([  1.78499569e-04,   2.39032437e-04,   2.48136028e-04, ...,
         2.61404290e-04,   1.43176556e-04,   8.28821358e-05], dtype=float32)

In [25]:
test_y[0]

array([False, False, False, ..., False, False, False], dtype=bool)

In [23]:
p[0]

1938

In [22]:
y[0]

2603

In [46]:
def evaluate(preds, golds, k=5):
    """ 
        Returns precision, recall and f1 at a given k number of tags 
        
        Args:
            preds: values for predicted tags for ea instance
            golds: actual tag values, boolean vector of len #tags
    """
    
    prec = []
    rec = []
    any_matches = 0
    
    assert len(preds) == len(golds)
        
    for i in tqdm(range(len(preds))):
#         print()
#         print(i)
        p = preds[i]
        gold = golds[i]
        
        top_pred_args = np.argsort(p)[::-1][:k]
        g = np.where(gold)[0]
        
        if len(g) == 0: # should be excluded
            continue
        
#         print(g)
        
        # Compare with gold tweets
#         for t in top_pred_args:
#             print(t)
#             print(t in g)
        matches = sum(1 for t in top_pred_args if t in g)
        if matches > 0:
            any_matches += matches
            print("Total tag matches: {}".format(any_matches))
        prec.append(matches/k)
        rec.append(matches/len(g))
        
    # Compute averages, f1
    avg_p = np.mean(prec)
    avg_r = np.mean(rec)
    mean_f1 = 2 * avg_p * avg_r / (avg_p + avg_r)

    return avg_p, avg_r, mean_f1

In [50]:
for k in range(1,6):
    print(k)
    print(evaluate(pred, test_y, k=k))

1


HBox(children=(IntProgress(value=0, max=19614), HTML(value='')))

Total tag matches: 1
Total tag matches: 2

(0.00011366219595362583, 8.524664696521937e-05, 9.7424739388822135e-05)
2


HBox(children=(IntProgress(value=0, max=19614), HTML(value='')))

Total tag matches: 1
Total tag matches: 2
Total tag matches: 3
Total tag matches: 4
Total tag matches: 5
Total tag matches: 6
Total tag matches: 7

(0.00019890884291884518, 0.00034098658786087748, 0.00025125327526590973)
3


...

Total tag matches: 1
Total tag matches: 2
Total tag matches: 3
Total tag matches: 4
Total tag matches: 5
Total tag matches: 6
Total tag matches: 7
Total tag matches: 8
Total tag matches: 9

(0.00017049329393043874, 0.00042623323482609686, 0.0002435618484720553)
4


HBox(children=(IntProgress(value=0, max=19614), HTML(value='')))

Total tag matches: 1
Total tag matches: 2
Total tag matches: 3
Total tag matches: 4
Total tag matches: 5
Total tag matches: 6
Total tag matches: 7
Total tag matches: 8
Total tag matches: 9
Total tag matches: 10

(0.00014207774494203227, 0.00048306433280290975, 0.00021957469672859536)
5


HBox(children=(IntProgress(value=0, max=19614), HTML(value='')))

Total tag matches: 1
Total tag matches: 2
Total tag matches: 3
Total tag matches: 4
Total tag matches: 5
Total tag matches: 6
Total tag matches: 7
Total tag matches: 8
Total tag matches: 9
Total tag matches: 10
Total tag matches: 11
Total tag matches: 12
Total tag matches: 13

(0.00014776085473971359, 0.00065355762673334852, 0.00024102834461088032)


In [24]:
pre_count = 0

for i, p in enumerate(pred):
    args = np.argsort(p)[::-1]
    if test_y[i][args[0]] == 1.:
        pre_count += 1

pre_count / pred.shape[0]
        

0.2033408184035799

In [58]:
tag_counts = defaultdict(lambda: 0)

for y in test_y:
    targets = np.nonzero(y)[0]
    #print (targets)
    for t in targets:
        tag_counts[t] += 1
        
freq_tags = set()
for k, v in tag_counts.items():
    if v > 19000:
        freq_tags.add(k)
        
len(freq_tags)
#indices_tag[299]
#len(tag_counts)

0

In [61]:
mean_ranks = []

for i, p in enumerate(pred):
    ranks = []
    args = np.argsort(p)[::-1]
    #print (p[args], args)
    #break
    targets = np.nonzero(test_y[i])[0]
    for t in targets:
        ranks.append(np.nonzero(args == t)[0][0])
    mean_ranks.extend(ranks)

np.mean(np.asarray(mean_ranks))

3.8947458528740975

In [62]:
recalls = []

for i, p in enumerate(pred):
    recall_count = 0
    args = list(np.argsort(p)[::-1][:10])
    targets = np.nonzero(test_y[i])[0]
    if len(targets) == 0:
        continue
    num_tags = len(targets)
    for t in targets:
        if t in args:
            recall_count += 1
    #mean_ranks.append(np.mean(np.asarray(ranks)))
    recalls.append(recall_count / num_tags)

np.mean(np.asarray(recalls))

0.93907085507503907